In [33]:
#set the current directory for reading the .csv files more easily
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/kaggle')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
#import the necessary packages
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [35]:
#read the traing and testing data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [36]:
train_data

,id,date,speed
0,0,1/1/2017 0:00,43.002930
1,1,1/1/2017 1:00,46.118696
2,2,1/1/2017 2:00,44.294158
3,3,1/1/2017 3:00,41.067468
4,4,1/1/2017 4:00,46.448653
...,...,...,...
14001,14001,31/12/2018 12:00,19.865269
14002,14002,31/12/2018 15:00,17.820375
14003,14003,31/12/2018 16:00,12.501851
14004,14004,31/12/2018 18:00,15.979319


In [37]:
test_data

,id,date
0,0,1/1/2018 2:00
1,1,1/1/2018 5:00
2,2,1/1/2018 7:00
3,3,1/1/2018 8:00
4,4,1/1/2018 10:00
...,...,...
3499,3499,31/12/2018 17:00
3500,3500,31/12/2018 19:00
3501,3501,31/12/2018 21:00
3502,3502,31/12/2018 22:00


In [38]:
#use the wwo-hist pakage to retrieve historical weather data
!pip install wwo-hist

In [22]:
#retrieving weather data for hongkong
from wwo_hist import retrieve_hist_data
import urllib.request

frequency = 24
start_date = '01-Jan-2017'
end_date = '31-DEC-2018'
api_key = '8b9dd8eb5f174b05b0a143348200412'
location_list = ['hongkong']

hist_weather_data = retrieve_hist_data(api_key, location_list, start_date, end_date, frequency = 24, export_csv = True, store_df = True)



Retrieving weather data for hongkong


Currently retrieving data for hongkong: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:00:00.719310
Currently retrieving data for hongkong: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:00:01.356933
Currently retrieving data for hongkong: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.042429
Currently retrieving data for hongkong: from 2017-04-01 to 2017-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.773515
Currently retrieving data for hongkong: from 2017-05-01 to 2017-05-31
Time elapsed (hh:mm:ss.ms) 0:00:03.452865
Currently retrieving data for hongkong: from 2017-06-01 to 2017-06-30
Time elapsed (hh:mm:ss.ms) 0:00:04.127652
Currently retrieving data for hongkong: from 2017-07-01 to 2017-07-31
Time elapsed (hh:mm:ss.ms) 0:00:04.844858
Currently retrieving data for hongkong: from 2017-08-01 to 2017-08-31
Time elapsed (hh:mm:ss.ms) 0:00:05.535242
Currently retrieving data for hongkong: from 2017-09-01 to 2017

In [23]:
#extarct weather features
dateparse = lambda dates: datetime.strptime(dates, '%Y-%m-%d')
df_weather = pd.read_csv('hongkong.csv', parse_dates=['date_time'], index_col='date_time', date_parser=dateparse)
weather_features = df_weather[['tempC','visibility','winddirDegree','windspeedKmph','humidity','cloudcover', 'WindChillC']]
weather_features

,tempC,visibility,winddirDegree,windspeedKmph,humidity,cloudcover,WindChillC
date_time,,,,,,,
2017-01-01,21,10,76,16,80,30,19
2017-01-02,22,10,77,10,80,11,21
2017-01-03,22,10,81,14,81,8,21
2017-01-04,22,10,80,15,82,8,21
2017-01-05,22,10,72,9,83,42,21
...,...,...,...,...,...,...,...
2018-12-27,22,10,39,12,66,43,22
2018-12-28,20,10,36,19,61,27,18
2018-12-29,14,10,21,20,62,44,12


In [24]:

def get_hour(s):
  s=str(s)
  list = re.split(' |:',s)
  return list[1]

def get_month(s):
  s=str(s)
  list = re.split('/',s)
  return list[1]

def get_year(s):
  s=str(s)
  list = re.split('/| ',s)
  return list[2]

def get_day(s):
  s=str(s)
  list = re.split('/| ',s)
  return list[0]

def get_week(s):
  s=str(s)
  list = re.split(' ', s)
  week = datetime.strptime(list[0],"%d/%m/%Y").weekday()
  return str(week)


holidays = ['2017-01-02','2017-01-28','2017-01-30','2017-01-31','2017-04-04','2017-04-14','2017-04-15','2017-04-17','2017-05-01','2017-05-03','2017-05-30','2017-07-01','2017-10-02','2017-10-05','2017-10-28','2017-12-25','2017-12-26','2018-01-01','2018-02-16','2018-02-17','2018-02-18','2018-03-30','2018-03-31','2018-04-02','5/4/2018','2018-05-01','2018-05-22','2018-06-18','2018-07-02','2018-09-25','2018-10-01','2018-10-17','2018-12-25','2018-12-26']

def get_holiday(s):
  s=str(s)
  list = re.split(' ', s)
  days = str(datetime.strptime(list[0],"%d/%m/%Y"))
  day = re.split(' ', days)[0]
  if day in holidays:
    return 1
  return 0

In [25]:
#extract time features for training data
train_data["hour"] = train_data["date"].apply(lambda x : get_hour(x))
train_data["month"] = train_data["date"].apply(lambda x : get_month(x))
train_data["day"] = train_data["date"].apply(lambda x : get_day(x))
train_data["year"] = train_data["date"].apply(lambda x : get_year(x))
train_data["weekday"] = train_data["date"].apply(lambda x: get_week(x))
train_data["holiday"] = train_data["date"].apply(lambda x: get_holiday(x))
train_data

,id,date,speed,hour,month,day,year,weekday,holiday
0,0,1/1/2017 0:00,43.002930,0,1,1,2017,6,0
1,1,1/1/2017 1:00,46.118696,1,1,1,2017,6,0
2,2,1/1/2017 2:00,44.294158,2,1,1,2017,6,0
3,3,1/1/2017 3:00,41.067468,3,1,1,2017,6,0
4,4,1/1/2017 4:00,46.448653,4,1,1,2017,6,0
...,...,...,...,...,...,...,...,...,...
14001,14001,31/12/2018 12:00,19.865269,12,12,31,2018,0,0
14002,14002,31/12/2018 15:00,17.820375,15,12,31,2018,0,0
14003,14003,31/12/2018 16:00,12.501851,16,12,31,2018,0,0
14004,14004,31/12/2018 18:00,15.979319,18,12,31,2018,0,0


In [26]:
#add the weather features
tempC = []
visibility = []
winddirDegree = []
windspeedKmph = []
humidity = []
cloudcover = []
WindChillC = []

# transform the type from date to datetime
for i in train_data['date']:
  string_date = str(i)
  string_date=re.split(" ",string_date)[0]
  date = datetime.strptime(string_date, '%d/%m/%Y')
  tempC.append(weather_features['tempC'][date])
  visibility.append(weather_features['visibility'][date])
  winddirDegree.append(weather_features['winddirDegree'][date])
  windspeedKmph.append(weather_features['windspeedKmph'][date])
  humidity.append(weather_features['humidity'][date])
  cloudcover.append(weather_features['cloudcover'][date])
  WindChillC.append(weather_features['WindChillC'][date])

train_data['tempC'] = tempC
train_data['visibility'] = visibility
train_data['winddirDegree'] = winddirDegree
train_data['windspeedKmph'] = windspeedKmph
train_data['humidity'] = humidity
train_data['cloudcover'] = cloudcover
train_data['WindChillC'] = WindChillC
train_data

,id,date,speed,hour,month,day,year,weekday,holiday,tempC,visibility,winddirDegree,windspeedKmph,humidity,cloudcover,WindChillC
0,0,1/1/2017 0:00,43.002930,0,1,1,2017,6,0,21,10,76,16,80,30,19
1,1,1/1/2017 1:00,46.118696,1,1,1,2017,6,0,21,10,76,16,80,30,19
2,2,1/1/2017 2:00,44.294158,2,1,1,2017,6,0,21,10,76,16,80,30,19
3,3,1/1/2017 3:00,41.067468,3,1,1,2017,6,0,21,10,76,16,80,30,19
4,4,1/1/2017 4:00,46.448653,4,1,1,2017,6,0,21,10,76,16,80,30,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14001,14001,31/12/2018 12:00,19.865269,12,12,31,2018,0,0,12,10,138,18,69,79,10
14002,14002,31/12/2018 15:00,17.820375,15,12,31,2018,0,0,12,10,138,18,69,79,10
14003,14003,31/12/2018 16:00,12.501851,16,12,31,2018,0,0,12,10,138,18,69,79,10
14004,14004,31/12/2018 18:00,15.979319,18,12,31,2018,0,0,12,10,138,18,69,79,10


In [27]:
traindata_cleaned = train_data.drop(["date"], axis=1)
traindata_cleaned

,id,speed,hour,month,day,year,weekday,holiday,tempC,visibility,winddirDegree,windspeedKmph,humidity,cloudcover,WindChillC
0,0,43.002930,0,1,1,2017,6,0,21,10,76,16,80,30,19
1,1,46.118696,1,1,1,2017,6,0,21,10,76,16,80,30,19
2,2,44.294158,2,1,1,2017,6,0,21,10,76,16,80,30,19
3,3,41.067468,3,1,1,2017,6,0,21,10,76,16,80,30,19
4,4,46.448653,4,1,1,2017,6,0,21,10,76,16,80,30,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14001,14001,19.865269,12,12,31,2018,0,0,12,10,138,18,69,79,10
14002,14002,17.820375,15,12,31,2018,0,0,12,10,138,18,69,79,10
14003,14003,12.501851,16,12,31,2018,0,0,12,10,138,18,69,79,10
14004,14004,15.979319,18,12,31,2018,0,0,12,10,138,18,69,79,10


In [28]:
traindata_cleaned.to_csv("trainingdata_cleaned-1.csv", index=False)

In [29]:
#extract time features for testing data,the proxess is the same as traing data
test_data["hour"] = test_data["date"].apply(lambda x : get_hour(x))
test_data["month"] = test_data["date"].apply(lambda x : get_month(x))
test_data["day"] = test_data["date"].apply(lambda x : get_day(x))
test_data["year"] = test_data["date"].apply(lambda x : get_year(x))
test_data["weekday"] = test_data["date"].apply(lambda x: get_week(x))
test_data["holiday"] = test_data["date"].apply(lambda x: get_holiday(x))
test_data

,id,date,hour,month,day,year,weekday,holiday
0,0,1/1/2018 2:00,2,1,1,2018,0,1
1,1,1/1/2018 5:00,5,1,1,2018,0,1
2,2,1/1/2018 7:00,7,1,1,2018,0,1
3,3,1/1/2018 8:00,8,1,1,2018,0,1
4,4,1/1/2018 10:00,10,1,1,2018,0,1
...,...,...,...,...,...,...,...,...
3499,3499,31/12/2018 17:00,17,12,31,2018,0,0
3500,3500,31/12/2018 19:00,19,12,31,2018,0,0
3501,3501,31/12/2018 21:00,21,12,31,2018,0,0
3502,3502,31/12/2018 22:00,22,12,31,2018,0,0


In [30]:
#add the weather features
tempC = []
visibility = []
winddirDegree = []
windspeedKmph = []
humidity = []
cloudcover = []
WindChillC = []

# transform the type from date to datetime
for i in test_data['date']:
  string_date = str(i)
  string_date=re.split(" ",string_date)[0]
  date = datetime.strptime(string_date, '%d/%m/%Y')
  tempC.append(weather_features['tempC'][date])
  visibility.append(weather_features['visibility'][date])
  winddirDegree.append(weather_features['winddirDegree'][date])
  windspeedKmph.append(weather_features['windspeedKmph'][date])
  humidity.append(weather_features['humidity'][date])
  cloudcover.append(weather_features['cloudcover'][date])
  WindChillC.append(weather_features['WindChillC'][date])

test_data['tempC'] = tempC
test_data['visibility'] = visibility
test_data['winddirDegree'] = winddirDegree
test_data['windspeedKmph'] = windspeedKmph
test_data['humidity'] = humidity
test_data['cloudcover'] = cloudcover
test_data['WindChillC'] = WindChillC
test_data

,id,date,hour,month,day,year,weekday,holiday,tempC,visibility,winddirDegree,windspeedKmph,humidity,cloudcover,WindChillC
0,0,1/1/2018 2:00,2,1,1,2018,0,1,19,10,65,12,63,23,18
1,1,1/1/2018 5:00,5,1,1,2018,0,1,19,10,65,12,63,23,18
2,2,1/1/2018 7:00,7,1,1,2018,0,1,19,10,65,12,63,23,18
3,3,1/1/2018 8:00,8,1,1,2018,0,1,19,10,65,12,63,23,18
4,4,1/1/2018 10:00,10,1,1,2018,0,1,19,10,65,12,63,23,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,3499,31/12/2018 17:00,17,12,31,2018,0,0,12,10,138,18,69,79,10
3500,3500,31/12/2018 19:00,19,12,31,2018,0,0,12,10,138,18,69,79,10
3501,3501,31/12/2018 21:00,21,12,31,2018,0,0,12,10,138,18,69,79,10
3502,3502,31/12/2018 22:00,22,12,31,2018,0,0,12,10,138,18,69,79,10


In [31]:
test_data["speed"] = 0
testdata_cleaned = test_data.drop(["date"], axis=1)
testdata_cleaned

,id,hour,month,day,year,weekday,holiday,tempC,visibility,winddirDegree,windspeedKmph,humidity,cloudcover,WindChillC,speed
0,0,2,1,1,2018,0,1,19,10,65,12,63,23,18,0
1,1,5,1,1,2018,0,1,19,10,65,12,63,23,18,0
2,2,7,1,1,2018,0,1,19,10,65,12,63,23,18,0
3,3,8,1,1,2018,0,1,19,10,65,12,63,23,18,0
4,4,10,1,1,2018,0,1,19,10,65,12,63,23,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,3499,17,12,31,2018,0,0,12,10,138,18,69,79,10,0
3500,3500,19,12,31,2018,0,0,12,10,138,18,69,79,10,0
3501,3501,21,12,31,2018,0,0,12,10,138,18,69,79,10,0
3502,3502,22,12,31,2018,0,0,12,10,138,18,69,79,10,0


In [32]:
testdata_cleaned.to_csv("testingdata_cleaned-1.csv", index=False)